In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from util_funcs import process_data, split_data
from neural_networks import SimpleNN, ComplexNN

In [2]:
df = pd.read_csv('/Users/niahodges/Downloads/Spring 2024/Practicum in Data Analytics and Statistics/project/temp-data/cps_ALL.csv')

In [3]:
df

,YEAR,SERIAL,MONTH,CPSID,ASECFLAG,HFLAG,ASECWTH,PERNUM,CPSIDV,CPSIDP,...,SCHLCOLL,DIFFANY,WKSWORK1,WKSWORK2,UHRSWORKLY,WKSUNEM1,WKSUNEM2,INCTOT,ADJGINC,TAXINC
0,2014,1,3,20140102878600,1,0.0,569.48,1,201401028786011,20140102878601,...,0,1,52,6,45,99,9,101500,170456,148056
1,2014,1,3,20140102878600,1,0.0,569.48,2,201401028786021,20140102878602,...,0,1,52,6,45,99,9,77300,0,0
2,2014,1,3,20140102878600,1,0.0,569.48,3,201401028786031,20140102878603,...,5,1,52,6,40,99,9,29003,29003,8353
3,2014,2,3,20131202917500,1,0.0,564.01,1,201312029175011,20131202917501,...,5,1,48,5,40,0,0,26800,9294,0
4,2014,2,3,20131202917500,1,0.0,564.01,2,201312029175021,20131202917502,...,1,1,16,2,10,0,0,19247,2447,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851268,2023,88977,3,0,1,NaN,413.20,1,0,0,...,5,1,52,6,40,99,9,50215,90415,64515
851269,2023,88977,3,0,1,NaN,413.20,2,0,0,...,5,1,52,6,40,99,9,40200,0,0
851270,2023,88978,3,0,1,NaN,416.37,1,0,0,...,0,1,52,6,40,99,9,33113,111205,85305
851271,2023,88978,3,0,1,NaN,416.37,2,0,0,...,0,1,52,6,40,99,9,78092,0,0


In [4]:
df_temp1 = df.copy()
df_gender = process_data(df_temp1,'gender')
df_temp2 = df.copy()
df_noGender = process_data(df_temp2,'no-gender')

In [5]:
X_train_G, X_test_G, y_train_G, y_test_G = split_data(df_gender)
X_train_NG, X_test_NG, y_train_NG, y_test_NG = split_data(df_noGender)

In [6]:
weights_filename = 'logs/feature_weights.txt'

# Gender inclusive models

In [ ]:
nn1_G = SimpleNN(input_size=X_train_G.shape[1], hidden_size=10, output_size=1)
nn2_G = ComplexNN(input_size=X_train_G.shape[1], 
                  hidden_sizes=[X_train_G.shape[1], 45, 22, 11], 
                  output_size=1)

## Simple NN

In [ ]:
input_size=X_train_G.shape[1]
hidden_size=10
output_size=1

weights_input_hidden = np.random.randn(input_size, hidden_size)
weights_hidden_output = np.random.randn(hidden_size, output_size)

In [ ]:
def forward(inputs):
    # Perform forward pass
    hidden = np.dot(inputs, weights_input_hidden)
    hidden_activation = sigmoid(hidden)
    output = np.dot(hidden_activation, weights_hidden_output)
    return output, hidden_activation              ### UPDATE ###
    
def sigmoid(x):
    # Sigmoid activation function
    return 1 / (1 + np.exp(-x))
    
def get_weights():
    return weights_input_hidden, weights_hidden_output

In [ ]:
mse_history = []
mae_history = []
X_train = X_train_G
y_train = y_train_G
learning_rate=0.01
epochs = 100

print('Training simple NN...')

for epoch in range(epochs):
        # Forward pass
        print(f'Forward pass for epoch {epoch+1}')
        output, hidden_activation = forward(X_train)
        output = output.reshape(-1)         ### UPDATE ###

        # Calculate MSE and MAE
        print(f'Calculating MSE and MAE for epoch {epoch+1}')
        if np.isnan(output).any():                 ### UPDATE ###
            output = np.nan_to_num(output)
        mse = mean_squared_error(y_train, output)
        mae = mean_absolute_error(y_train, output)

        # Store MSE and MAE
        mse_history.append(mse)
        mae_history.append(mae)

        # Backpropagation
        print(f'Backpropogation for epoch {epoch+1}')
        print('------------------DEBUGGING-------------------')
        print(f'Output shape: {output.shape}')
        print(f'Y_train shape: {y_train.shape}')
        error = output - y_train
        print("Shapes:")
        delta_output = error
        delta_output = delta_output.reshape(-1, 1)        ### UPDATE ###
        print("delta_output:", delta_output.shape)
        output = output.reshape(-1, 1)         ### UPDATE ###
        delta_hidden = np.dot(delta_output, weights_hidden_output.T) * (output * (1 - output))
        print("delta_hidden:", delta_hidden.shape)
        print('hidden_size:', hidden_size)
        print('-----------------------------------------------')

        # Update weights
        print(f'Updating weights for epoch {epoch+1}')
        weights_hidden_output -= learning_rate * np.dot(hidden_activation.T, delta_output)
        weights_input_hidden -= learning_rate * np.dot(X_train.T, delta_hidden)

        if epoch % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, MSE: {mse}, MAE: {mae}")
            
print('Done training simple NN!')

In [ ]:
nn1_input_weights_G, nn1_output_weights_G = get_weights()

In [ ]:
temp_nn1_input_weights_G = []
for i in range(len(nn1_input_weights_G)):
    temp_nn1_input_weights_G.append(nn1_input_weights_G[i][0])

In [ ]:
temp_nn1_output_weights_G = []
for i in range(len(nn1_output_weights_G)):
    temp_nn1_output_weights_G.append(nn1_output_weights_G[i][0])

In [ ]:
learning_history_G = {'epoch':range(1,101),
                      # 'mse_lasso':mseHistory_lasso_G,
                      # 'mse_ridge':mseHistory_ridge_G,
                      # 'mse_svr':mseHistory_svr_G,
                      'mse_nn1':mse_history,
                      # 'mse_nn2':mseHistory_nn2_G,
                      # 'mae_lasso':maeHistory_lasso_G,
                      # 'mae_ridge':maeHistory_ridge_G,
                      # 'mae_svr':maeHistory_svr_G,
                      'mae_nn1':mae_history,
                      # 'mae_nn2':maeHistory_nn2_G
                     }

In [ ]:
df_learning_history_G = pd.DataFrame(learning_history_G)
csv_filename = 'logs/training_history_SimpleNN_gender.csv'
df_learning_history_G.to_csv(csv_filename, index=False)

In [ ]:
nn1_Gender_inputWeights_df = pd.DataFrame({'input_weights': temp_nn1_input_weights_G})
nn1_Gender_inputWeights_df.to_csv('logs/nn1_G_inputWeights.csv', index=False)
nn1_Gender_outputWeights_df = pd.DataFrame({'output_weights': temp_nn1_output_weights_G})
nn1_Gender_outputWeights_df.to_csv('logs/nn1_G_outputWeights.csv', index=False)

# Gender exclusive models

## Simple NN

In [7]:
input_size=X_train_NG.shape[1]
hidden_size=10
output_size=1

weights_input_hidden = np.random.randn(input_size, hidden_size)
weights_hidden_output = np.random.randn(hidden_size, output_size)

In [8]:
def forward(inputs):
    # Perform forward pass
    hidden = np.dot(inputs, weights_input_hidden)
    hidden_activation = sigmoid(hidden)
    output = np.dot(hidden_activation, weights_hidden_output)
    return output, hidden_activation              ### UPDATE ###
    
def sigmoid(x):
    # Sigmoid activation function
    return 1 / (1 + np.exp(-x))
    
def get_weights():
    return weights_input_hidden, weights_hidden_output

In [9]:
mse_history = []
mae_history = []
X_train = X_train_NG
y_train = y_train_NG
learning_rate=0.01
epochs = 100

print('Training simple NN...')

for epoch in range(epochs):
        # Forward pass
        print(f'Forward pass for epoch {epoch+1}')
        output, hidden_activation = forward(X_train)
        output = output.reshape(-1)         ### UPDATE ###

        # Calculate MSE and MAE
        print(f'Calculating MSE and MAE for epoch {epoch+1}')
        if np.isnan(output).any():                 ### UPDATE ###
            output = np.nan_to_num(output)
        mse = mean_squared_error(y_train, output)
        mae = mean_absolute_error(y_train, output)

        # Store MSE and MAE
        mse_history.append(mse)
        mae_history.append(mae)

        # Backpropagation
        print(f'Backpropogation for epoch {epoch+1}')
        print('------------------DEBUGGING-------------------')
        print(f'Output shape: {output.shape}')
        print(f'Y_train shape: {y_train.shape}')
        error = output - y_train
        print("Shapes:")
        delta_output = error
        delta_output = delta_output.reshape(-1, 1)        ### UPDATE ###
        print("delta_output:", delta_output.shape)
        output = output.reshape(-1, 1)         ### UPDATE ###
        delta_hidden = np.dot(delta_output, weights_hidden_output.T) * (output * (1 - output))
        print("delta_hidden:", delta_hidden.shape)
        print('hidden_size:', hidden_size)
        print('-----------------------------------------------')

        # Update weights
        print(f'Updating weights for epoch {epoch+1}')
        weights_hidden_output -= learning_rate * np.dot(hidden_activation.T, delta_output)
        weights_input_hidden -= learning_rate * np.dot(X_train.T, delta_hidden)

        if epoch % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, MSE: {mse}, MAE: {mae}")
            
print('Done training simple NN!')

Training simple NN...
Forward pass for epoch 1
Calculating MSE and MAE for epoch 1
Backpropogation for epoch 1
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
Updating weights for epoch 1
Epoch 1/100, MSE: 4735615355.196975, MAE: 53242.49165843456
Forward pass for epoch 2
Calculating MSE and MAE for epoch 2
Backpropogation for epoch 2
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
Updating weights for epoch 2
Forward pass for epoch 3
Calculating MSE and MAE for epoch 3
Backpropogation for epoch 3
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692

/var/folders/y_/y10p46q954j0sbmbks47d_xm0000gq/T/ipykernel_10215/381880660.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
/var/folders/y_/y10p46q954j0sbmbks47d_xm0000gq/T/ipykernel_10215/381880660.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Calculating MSE and MAE for epoch 4
Backpropogation for epoch 4
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
Updating weights for epoch 4
Forward pass for epoch 5
Calculating MSE and MAE for epoch 5
Backpropogation for epoch 5
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
Updating weights for epoch 5
Forward pass for epoch 6
Calculating MSE and MAE for epoch 6
Backpropogation for epoch 6
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
Updating weights for epoch 6
Forward 

/var/folders/y_/y10p46q954j0sbmbks47d_xm0000gq/T/ipykernel_10215/381880660.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
/var/folders/y_/y10p46q954j0sbmbks47d_xm0000gq/T/ipykernel_10215/381880660.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Calculating MSE and MAE for epoch 8
Backpropogation for epoch 8
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
Updating weights for epoch 8
Forward pass for epoch 9
Calculating MSE and MAE for epoch 9
Backpropogation for epoch 9
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
Updating weights for epoch 9
Forward pass for epoch 10
Calculating MSE and MAE for epoch 10
Backpropogation for epoch 10
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
Updating weights for epoch 10
Forw

...

Calculating MSE and MAE for epoch 12
Backpropogation for epoch 12
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
Updating weights for epoch 12
Forward pass for epoch 13
Calculating MSE and MAE for epoch 13
Backpropogation for epoch 13
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
Updating weights for epoch 13
Forward pass for epoch 14
Calculating MSE and MAE for epoch 14
Backpropogation for epoch 14
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
Updating weights for epoch 

/var/folders/y_/y10p46q954j0sbmbks47d_xm0000gq/T/ipykernel_10215/381880660.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
/var/folders/y_/y10p46q954j0sbmbks47d_xm0000gq/T/ipykernel_10215/381880660.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Forward pass for epoch 16
Calculating MSE and MAE for epoch 16
Backpropogation for epoch 16
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
Updating weights for epoch 16
Forward pass for epoch 17
Calculating MSE and MAE for epoch 17
Backpropogation for epoch 17
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
Updating weights for epoch 17
Forward pass for epoch 18
Calculating MSE and MAE for epoch 18
Backpropogation for epoch 18
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
U

/var/folders/y_/y10p46q954j0sbmbks47d_xm0000gq/T/ipykernel_10215/381880660.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
/var/folders/y_/y10p46q954j0sbmbks47d_xm0000gq/T/ipykernel_10215/2973302574.py:38: RuntimeWarning: overflow encountered in multiply
  delta_hidden = np.dot(delta_output, weights_hidden_output.T) * (output * (1 - output))


Forward pass for epoch 19
Calculating MSE and MAE for epoch 19
Backpropogation for epoch 19
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
Updating weights for epoch 19
Forward pass for epoch 20
Calculating MSE and MAE for epoch 20
Backpropogation for epoch 20
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
Updating weights for epoch 20
Forward pass for epoch 21
Calculating MSE and MAE for epoch 21
Backpropogation for epoch 21
------------------DEBUGGING-------------------
Output shape: (444692,)
Y_train shape: (444692,)
Shapes:
delta_output: (444692, 1)
delta_hidden: (444692, 10)
hidden_size: 10
-----------------------------------------------
U

In [10]:
nn1_input_weights_NG, nn1_output_weights_NG = get_weights()

In [11]:
temp_nn1_input_weights_NG = []
for i in range(len(nn1_input_weights_NG)):
    temp_nn1_input_weights_NG.append(nn1_input_weights_NG[i][0])

In [12]:
temp_nn1_output_weights_NG = []
for i in range(len(nn1_output_weights_NG)):
    temp_nn1_output_weights_NG.append(nn1_output_weights_NG[i][0])

In [13]:
learning_history_NG = {'epoch':range(1,101),
                      # 'mse_lasso':mseHistory_lasso_G,
                      # 'mse_ridge':mseHistory_ridge_G,
                      # 'mse_svr':mseHistory_svr_G,
                      'mse_nn1':mse_history,
                      # 'mse_nn2':mseHistory_nn2_G,
                      # 'mae_lasso':maeHistory_lasso_G,
                      # 'mae_ridge':maeHistory_ridge_G,
                      # 'mae_svr':maeHistory_svr_G,
                      'mae_nn1':mae_history,
                      # 'mae_nn2':maeHistory_nn2_G
                     }

In [14]:
df_learning_history_NG = pd.DataFrame(learning_history_NG)
csv_filename = 'logs/training_history_SimpleNN_no-gender.csv'
df_learning_history_NG.to_csv(csv_filename, index=False)

In [15]:
nn1_NoGender_inputWeights_df = pd.DataFrame({'input_weights': temp_nn1_input_weights_NG})
nn1_NoGender_inputWeights_df.to_csv('logs/nn1_NG_inputWeights.csv', index=False)
nn1_NoGender_outputWeights_df = pd.DataFrame({'output_weights': temp_nn1_output_weights_NG})
nn1_NoGender_outputWeights_df.to_csv('logs/nn1_NG_outputWeights.csv', index=False)

# End